# Data 401 Project 1
Team Rum


In [1]:
# Imports

import pandas as pd
import numpy as np
import sys
from sklearn.linear_model import LinearRegression

In [2]:
# Read the data and setup training sets

df = pd.read_csv("data/aggdata08.csv")

df.dropna(inplace=True)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
# shuffle
df = df.sample(frac=1)
X = df.drop(['volume_sold_l', 'volume_per_cap'], axis=1)
X['intercept'] = 1
Y = df['volume_per_cap'].copy()

In [5]:
def generate_model(data, y, features):
    """
    Generate linear regression model coefficients
    
    Params:
        data -- a pandas dataframe containing data (assumed to contain intercept 
                                                    column and no response variable)
        y -- a pandas series containing response variable
        features -- a list of strings containing which features to include in model
                    (do not include 'intercept', it will be automatically added)
                    
    Returns:
        A numpy array containing model coefficients
    """
    
    features = ["intercept"] + features
    mat_data = data.as_matrix(features)
    betas = (np.matmul(np.matmul(np.linalg.inv(np.matmul(mat_data.T, mat_data)), mat_data.T), y.as_matrix()))
    return betas


def calc_bic(data, y, coef, features):
    """
    Calculate BIC metric for a model
    
    Params:
        data -- a pandas dataframe containing data (assumed to contain intercept 
                                                    column and no response variable)
        y -- a pandas series containing response variable
        coef -- numpy array containing model coefficients
        features -- a list of strings containing which features to include in model
                    (do not include 'intercept', it will be automatically added)
        
    Returns:
        A float representing BIC metric
    """
    
    features = ["intercept"] + features
    mat_dat = data.as_matrix(features)
    
    n = len(data)
    var1 = np.matmul(mat_dat, coef.T)    
    residuals = y.as_matrix() - var1
    sse = np.sum(residuals**2)
    sst = y.var() * len(y)
    #print("R^2 = " + str(1 - sse/sst))

    return sse/y.var() + (len(features)-1) * np.log(n)
    
    
def calc_accuracy(X_test, y, coef, features):
    """
    Calculate accuracy metric for a model
    
    Params:
        X_test -- a pandas dataframe containing data (assumed to contain intercept 
                                                      column and no response variable)
        y -- a pandas series containing response variable
        coef -- numpy array containing model coefficients
        features -- a list of strings containing which features to include in model
                    (do not include 'intercept', it will be automatically added)
        
    Returns:
        A float representing accuracy metric
    """
    
    features = ["intercept"] + features
    mat_dat = X_test.as_matrix(features)
    
    n = len(X_test)
    var1 = np.matmul(mat_dat, coef.T)
    residuals = y.as_matrix() - var1
    sst = y.var() * len(y)
    sse = np.sum(residuals**2) 
    #print("R^2 = " + str(1 - sse/sst))
    
    return sse/n

In [6]:
# Forward Stepwise for BIC minimization
features = list(X.columns[3:-1])
features = list(set(features) - set(['county_bin_quartile1','county_bin_quartile2','county_bin_quartile3','county_bin_quartile4','Seasons_Winter','Seasons_Summer','Seasons_Spring']))

last_bic = sys.maxsize    # Last model prediction accuracy
model_bic = sys.maxsize   # Current model prediction accuracy
interp_features = []     # Current features in model
target_feature = None     # Current best feature
best_coefs = None

while len(interp_features) < len(features):
    for feature in list(set(features) - set(interp_features)):
        
        # Generate model
        try_features = interp_features + [feature]
        coefs = generate_model(X, Y, try_features)
        
        # Calculate bic
        cur_bic = calc_bic(X, Y, coefs, try_features)
        
        # Save feature if bic lower than current
        if last_bic is None or cur_bic < last_bic:
            last_bic = cur_bic
            target_feature = feature
            
    # If the best feature improves the model, add it
    if last_bic < model_bic and abs(last_bic-model_bic) > 0.03*model_bic:
        interp_features.append(target_feature)
        model_bic = last_bic
        best_coefs = coefs
        print("Chose: " + target_feature + " (" + str(model_bic) + ")")
        
    # Otherwise we are done
    else:
        print("Found Optimal Model, (%d features of %d)" % (len(interp_features), len(features)))
        print(interp_features)
        print(best_coefs)
        break


/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Chose: avg_vendor_size (10014.204262971787)
Chose: num_weeks (8912.813986011446)
Chose: average_HH_size (8065.799680824065)
Chose: rum_bottles (7663.526096743773)
Chose: population (7118.6012233350075)
Chose: male_over_21_prop (6805.55116395802)
Chose: avg_bottle_price (6557.636221740147)
Found Optimal Model, (7 features of 30)
['avg_vendor_size', 'num_weeks', 'average_HH_size', 'rum_bottles', 'population', 'male_over_21_prop', 'avg_bottle_price']
[ 1.27359756e+00  3.00627782e-04  2.57810772e-04 -3.03295757e-01
  3.95469173e-05 -1.56882787e-06 -1.47616723e+00  6.60753284e-03]


In [7]:
# Forward Stepwise for prediction error minimization
features = list(X.columns[3:-1])

last_err = sys.maxsize    # Last model prediction accuracy
model_err = sys.maxsize   # Current model prediction accuracy
pred_features = []     # Current features in model
target_feature = None     # Current best feature
best_coefs = None

while len(pred_features) < len(features):
    for feature in list(set(features) - set(pred_features)):
        
        # Cross validate
        folds = 5
        sp = len(X)/folds
        err = []
        for i in range(folds):
            lower = int(sp * i)
            upper = int(sp * (i+1))
            test_x = X.iloc[lower:upper]
            test_y = Y.iloc[lower:upper]
            train_x = pd.concat((X.iloc[:lower], X.iloc[upper:]))
            train_y = pd.concat((Y.iloc[:lower], Y.iloc[upper:]))
            
            # Generate model
            try_features = pred_features + [feature]
            coefs = generate_model(train_x, train_y, try_features)
        
            # Calculate predction error
            err += [calc_accuracy(test_x, test_y, coefs, try_features)]
            
        cur_err = sum(err)/len(err)
        
        # Save feature if error lower than current
        if last_err is None or cur_err < last_err:
            last_err = cur_err
            target_feature = feature
            best_coefs = coefs
    
    # If the best feature improves the model, add it
    if last_err < model_err:
        pred_features.append(target_feature)
        model_err = last_err
        print("Chose: " + target_feature + " (" + str(model_err) + ")")
        
    # Otherwise we are done
    else:
        print("Found Optimal Model, (%d features of %d)" % (len(pred_features), len(features)))
        print(pred_features)
        print(best_coefs)
        calc_accuracy(test_x, test_y, best_coefs, pred_features)
        break


/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/markelle/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Chose: avg_vendor_size (0.006892242137740964)
Chose: county_bin_quartile1 (0.006111216947842122)
Chose: county_bin_quartile2 (0.0054515465801426525)
Chose: county_bin_quartile3 (0.004869743207904271)
Chose: county_bin_quartile4 (0.0034987967840558005)
Chose: num_weeks (0.0031504603638142655)
Chose: head_of_household_prop (0.003058032602564909)
Chose: whiskey_bottles (0.0029341722726642034)
Chose: owned_housing (0.002596018392948796)
Chose: rum_bottles (0.0025037528890311522)
Chose: avg_bottle_price (0.002449544499803463)
Chose: average_HH_size (0.002427725841642008)
Chose: male_over_21_prop (0.002396357104232363)
Chose: isHoliday (0.002379070370224324)
Chose: pop_density (0.0023627961293756425)
Chose: white_prop (0.002344889108856533)
Chose: Seasons_Winter (0.002329876285848513)
Chose: over_65_prop (0.002315063084833253)
Chose: other_bottles (0.002303631924843917)
Chose: female_median_age (0.002298017437517505)
Chose: median_age (0.0022853449495383164)
Chose: tequila_bottles (0.0022795